# Test script

# TO DO

- добавить признак возраст бойца просто или до 30 инфа, так как этот индикатор должен менять зависимость между разницей в возрасте
    - куда делся признак угла
    - точно разница в возрасте не должна только определять исход 
    
- сделать так, чтобы убрать сообщение о mixed types df = pd.read_csv(conf['preprocess']['source_fights_fn'], dtype={'judges':str, 'judge1':str, 'judge2':str, 'judge3':str})
- касательно оптимизирации будто надо f1 оптимизировать при выборе границы, но тогда не понятно, как сравнивать свою вероятность с тотализаторной, допустим thresh 0.3 у тотализатора 0.5, как считать плохие и хорошие ставки
- валидация ставок:
    - разбиваем на train/val
    - усредненией прогнозов от всех источников
        - ставим на бой, который с большей средней вероятностью выиграет, но там, где больший коэффициент на этот бой
        - можно найти источник (букмекер или llm) наиболее верных прогнозов

- то же для футбольных матчей, кто лучше прогнозирует
    

- валидация прогнозов от llm
    - ссылки не битые и подтвержденные (deepseek, perplexity)
    - прогнозы близкие к значениям leon-а

- динамика качества ответов для chat-ов со временем (например, эрни), смотрим по отклонению от скачанных коэффициентов
- фильтруем, у кого отклонения сильно большие постоянно
- ставим на околовилочные бои, где большой разброс, при этом модель достаточно уверена
- [ставки на ufc](https://www.bestfightodds.com/)
    - [сканер вилок](https://vprognoze.ru/wiki/oddstorm-obzor-servisa/)
- словарь бойцов, надо подумать как его вести
    - алерт, если не совпадает имя или несколько аналогов, так как мог подтянуть другого
    - русском наименование
- нарисовать схему валидации и инференса
- Признаки на будущее:
        Средний рейтинг/рекорд последних соперников
        Победы над топ-10 бойцами
        Процент боёв против сильных соперников
        Разница в рангах, но для этого надо постоянно скачивать rankings

- как валидировать:

- у кого нет рейтингов может не предсказывать, так как откуда-то чемп мог прийти

- может не предсказывать для main event (так как сильно подвержены людским ставкам) или эти бои могут быть выгоднее, наоборот

- 
- подбор алгоритма и гиперпараметров в рамках него
- каждый event предсказывает правильно не менее половины из боев и зарабатывает на стратегии в целом
- можно проверить легкие стратегии, типа
    - ставим равномерно на тех, где коэффициент ниже
        - где уверенность выше порога некого
        - где коэфы в пределах ...
    - на того, кто в рейтингах выше
    - кто больше побеждал, в большем проценте случаев на того и ставим

- где можем, угол замещаем на реальный рейтинг и особенно там поведение смотрим
- проверить, что в красном тот, кто выше в рейтингах
      
- добавить флаг использовать ли редкие признаки, посмотреть какие группы и с какого периода заполняемость лучше, может и норм старые бои какие-то отрезать
- запустить старые тесты (рефакторинг кода) и добавить новые на фичи добавленные, смотреть, чтобы тянулись по цепочке (['dob_stat', 'height_stat', 'reach_stat', 'stance'])

  

- 
- посмотреть покрытие предсказаний (скорее всего, для бойцов с количеством менее скольки-то схваток нет предсказаний)
- разобрать вид завершения:
    - код:
     
      ```
      prep_df = pd.read_csv(conf['preprocess']['prep_fn'])
      prep_df.loc[(prep_df['judges'].str.len()<10), 'Details:'].unique()
      ```

      

- Если событие имеет вероятность P, то справедливый коэффициент должен быть:
    - Кэффициент = 1 / P, так как при многократном повторении ставок вы должны оставаться при своих деньгах.
    - положили x , x*1/p получим... при многократном повторении - x 
- Кейсы вероятностей для ставок (pr) и предсказаний (pr_pred), diff=pr - pr_pred, вердикт тотализатора:
    - переоценка - diff>0:
        - если pr_pred>50% то заработаем скорее всего, но мало, так как переоценена вероятность, может лучше рассмотреть недооценку (кейс 2) ставку на победу другого, так как она хоть и меньше 50%, но если сыграет заработаем больше ожидаемого
        - если pr_pred<50%, самый невыгодный случай, рассматриваем ставку на другого, так как там сымый выгодный - недооценка при pr_pred>50%
    - недооценка - diff<0
        - pr_pred>50% - лучший случай, на такой надо ставить
        - pr_pred<50% - хоть и меньше 50%, можно заработать
- функция полезности получает предсказания, ставки и selection, считает по нему выгоду, фильтруя бои для различных стратегий
- перевод американских коэфов:
    - "-" - сколько нужно поставить, чтобы заработать 100; 100/abs(x) + 1; x=-200, 0.5+1=1.5
    - "+" - сколько заработаем, если поставим 100; x/100+1; x=200, 2+1=3

- x - сумму положил, равномерно ее по боям распределил и сколько заработал
- n - количество угаданных боев
- receive=$1/n*\sum_icoef(x_i)*x_i - x$ 
- метрика оптимизации будет roc_auc + при ненулевом выигрыше со стратегией равномерного распределения по выигрышам. Функция от preds, y_true, coefs, strategy. При этом выигрыш временной, пусть x положили в начале и каждый этап мы что-то выигрываем/проигрываем а в конце в плюсе, если на какомто этапе менее n=5 боев оценивается, то добавляем его в предыдущий
  
- loss менять будем потом, можно подумать взвешенную регрессию (надо знать коэффициенты, только часть сможем взвесить)

# Description

Takedown (TD)
Knockdown (KD)